In [1]:
import funcs.scraper as scp

In [2]:
df = scp.download_carga_diaria(2000,2023,printer=True)

Lendo ano 2000...
Lendo ano 2001...
Lendo ano 2002...
Lendo ano 2003...
Lendo ano 2004...
Lendo ano 2005...
Lendo ano 2006...
Lendo ano 2007...
Lendo ano 2008...
Lendo ano 2009...
Lendo ano 2010...
Lendo ano 2011...
Lendo ano 2012...
Lendo ano 2013...
Lendo ano 2014...
Lendo ano 2015...
Lendo ano 2016...
Lendo ano 2017...
Lendo ano 2018...
Lendo ano 2019...
Lendo ano 2020...
Lendo ano 2021...
Lendo ano 2022...
Lendo ano 2023...


In [4]:
df.isna().sum()

id_reg         0
desc_reg       0
load_mwmed    61
dtype: int64